In [1]:
from IPython.html.services.config import ConfigManager
from IPython.paths import locate_profile
cm = ConfigManager(profile_dir=locate_profile(get_ipython().profile))
cm.update('livereveal', {
              'theme': 'solarized',
              'transition': 'slide',
              'start_slideshow_at': 'selected',
              'progress': 'true',
})

/usr/local/lib/python3.5/dist-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


{'background': '#ff0000',
 'height': 768,
 'progress': 'true',
 'scroll': 'true',
 'start_slideshow_at': 'selected',
 'theme': 'solarized',
 'transition': 'slide',
 'width': 1024}

<center>
<h2>Interpretation and connections</h2>
</center>

### Why does IL work?

A bit of theory (Goodman)

### Reinforcement learning

<p style="float: left;">
<ul style="float: left;">
<li>states defined via features</li>
<li>the agent is a classifier</li>
<li>rewards?</li>
</ul>
</p>

<a href="https://webdocs.cs.ualberta.ca/~sutton/book/ebook/node28.html"><img src="images/RL_sutton.png" style="width:44%; float: right;"></a>

**Inverse reinforcement learning**

- we have the expert policy (inferred from the gold standard in the training data)
- we infer the per-action reward function (rollin/out)				


Replacing the expert policy in LoLS with a random (sub-optimal) one is RL ([Chang et al., 2015](https://arxiv.org/pdf/1502.02206.pdf))

### Inverse reinforcement learning

- and learn a policy (classifier) to generalize to unseen data



### Bandit learning

Sokolov

### Semi/Unsupervised learning

Learning with non-decomposable loss functions means
- no need to know the correct actions
- learn to predict them in order to minimize the loss

<img src="images/tikz/semParse.png" style="width:100%;">

UNSEARN ([Daumé III, 2009](http://www.umiacs.umd.edu/~hal/docs/daume09unsearn.pdf)): Predict the structured output so that you can predict the input from it (auto-encoder!)

### Negative data sampling

<img src="images/tikz/posImitClassifierTraining.png" style="width:65%; align:left;">

- Expert action sequence → positive example
- All other action sequences → negative examples
- Using all negative examples inefficient

Imitation learning: generate negative samples around the expert 

A form of **Adversarial training** ([Ho and Ermon, 2016](https://arxiv.org/abs/1606.03476)) 

### Coaching

<p style="float: left;">If the optimal action is difficult<br>to predict, the coach teaches<br>a good one that is easier<br> (<a href="https://papers.nips.cc/paper/4545-imitation-learning-by-coaching.pdf">He et al., 2012</a>)</p> <a href="https://commons.wikimedia.org/wiki/File:US_Navy_091206-N-2013O-023_Sam_Givens,_a_player_for_the_Harlem_Ambassadors_basketball_team,_demonstrates_proper_dribbling_techniques_to_a_boy_during_a_basketball_camp_sponsored_by_Yokosuka%27s_Morale,_Welfare_and_Recreation_Youth.jpg"><img src="images/coaching.jpg" style="width:35%; float: right;"></a>

Expert: $\alpha^{\star}= \mathop{\arg \min}_{\alpha \in {\cal A}} L(S_t(\alpha,\pi^{\star}),\mathbf{y})$

Coach: $\alpha^{\star}= \mathop{\arg \min}_{\alpha \in {\cal A}} L(S_t(\alpha,\pi^{\star}),\mathbf{y}) - \lambda f(\alpha, \mathbf{x}, S_t)$

### Ranking

Jana Rao Doppa

<h3> What about Recurrent Neural Networks?</h3>

<img src="images/rnn.png" width="70%"  style="background:none;" />

<p>They face similar problems:
				<ul>
				<li>trained at the word rather than sentence level</li>
				<li>assume previous predictions are correct</li>
			</ul>
			</p>

### Imitation learning and RNNs

<img src="images/mixer.png" width="70%"  style="background:none;" />


- DAgger mixed rollins, similar to scheduled sampling ([Bengio et al., 2015](http://arxiv.org/abs/1506.03099))
- no rollouts, learn a  regressor to estimate action costs
- end-to-end back propagation through the sequence
- MIXER  (<a href="https://arxiv.org/abs/1511.06732">Ranzato et al., 2016</a>): Mix REINFORCE-ment learning with imitation: we have the expert policy!

### Actor critic

### Summary so far

- basic concepts
  - loss function decomposability
  - expert policy
- imitation learning
    - rollin/outs
    - DAgger algorithm
    - DAgger with rollouts and LoLS
- connections and interpretations

### After the break

- Applications:
  - dependency parsing
  - natural language generation
  - semantic parsing
- Practical advice
    - making things faster
    - debugging

<center>

<h1>Break!</h1>